In [20]:
import numpy as np 
import pandas as pd 
import torch

In [2]:
processed_train = pd.read_pickle('/home/amazon_ml_challenge/dataset/train_pickle.pkl')
processed_train = processed_train.sample(n=1000, random_state=42)

In [3]:
for i, row in processed_train.iterrows():
    bullet_points = row['BULLET_POINTS']
    bullet_points_words = " ".join(bullet_points).replace(",", "")
    processed_train.at[i, 'BULLET_POINTS'] = bullet_points_words

print(processed_train['BULLET_POINTS'])

1012021     O n e   o f   s i x    6    R a p i d   D e p...
1004588      T O Y   D O C T O R   K I T   F O R   K I D ...
1982634     M a n u f a c t u r e r   M o d e l    1 5 4 ...
314868      C o m p a t i b l e   W i t h    D e s i g n ...
1648868      b r  M a t e r i a l    R u b b e r    C l o...
                                 ...                        
683141      A s s o r t e d   c o l o r s   d o t   s t i...
1384310     S h e l l    9 0    c o t t o n  1 0    p o l...
2216952        C o z y   M a t e r i a l   M a d e   o f ...
1807140     S I Z E    1 1 o z    1 5 o z    P e r f e c ...
2135207     W H Y   B U Y   I T    T h i s   P r i n t e ...
Name: BULLET_POINTS, Length: 1000, dtype: object


In [4]:
processed_train['BULLET_POINTS'] = processed_train['BULLET_POINTS'].apply(lambda x: x.replace(" ", "").replace("\n", "").replace("\t", ""))
processed_train['BULLET_POINTS']  = processed_train['BULLET_POINTS'].str.replace(r'(?<=[a-z])(?=[A-Z0-9])', ' ')




<ipython-input-4-14b526c641c5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  processed_train['BULLET_POINTS']  = processed_train['BULLET_POINTS'].str.replace(r'(?<=[a-z])(?=[A-Z0-9])', ' ')


In [5]:
print(processed_train.head())

         PRODUCT_ID                                              TITLE  \
1012021     1024120  Rapid Deployment Force 'PARA NINJA' 12' Poseab...   
1004588     2693415  39 Pieces Doctor Kit for Kids, Educational Pla...   
1982634     1090054                   Frigidaire 154767801 Float Valve   
314868      1694624  Galaxy S8 Active Case, AMOOK Shockproof Hybrid...   
1648868     1779642       Anupreksha Men's Black Fashion Sandal - 6 UK   

                                             BULLET_POINTS  \
1012021  Oneofsix 6Rapid Deploymentfiguresproducedin 19...   
1004588  TOYDOCTORKITFORKIDTOROLEPLAYIncludingeveryimag...   
1982634  Manufacturer Model 154767801Genuine Replacemen...   
314868   Compatible With Designfor Samsung Galaxy S8Act...   
1648868  br Material Rubber Closure Slip Onbrbr Water R...   

                                               DESCRIPTION  PRODUCT_TYPE_ID  \
1012021                        twelve inch poseable figure             1225   
1004588  toy doctor kit 

In [6]:
processed_train['COMPLETE_DESCRIPTION'] = processed_train['TITLE'] + ' ' + processed_train['DESCRIPTION'] + ' ' + processed_train['BULLET_POINTS']


In [23]:
import transformers
from transformers import pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu"
# Load the GPT-2 model
model = pipeline('summarization',model = 'sshleifer/distilbart-cnn-12-6',device= device)


def get_max_length(input_length, fraction):
    max_length = int(input_length * fraction)
    return max_length


# Loop over the rows in the processed_train dataframe
for i, row in processed_train.iterrows():
    # Get the complete description
    complete_description = row['COMPLETE_DESCRIPTION']
    
    # Define the maximum length of the generated summary based on the length of the input text
    max_ratio = 0.3  # Set the maximum ratio between summary length and input text length
    max_length = max_length = get_max_length(input_length=int(len(complete_description)), fraction=0.4)
    min_length = int(len(complete_description) * 0.1)
    
    # Generate the summary using the distilbartmodel
    summary = model(complete_description, max_length=30, min_length=10, do_sample=True)

    processed_train.at[i, 'SUMMARY'] = summary[0]['summary_text']

    
    

Your max_length is set to 30, but you input_length is only 25. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)
Your max_length is set to 30, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 30, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 30, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 30, but you input_length is only 28. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)
Your max_length is set to 30, but you input_length is only 8. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 30, but you input_length is only 27. You might consider decreasi

In [28]:
processed_train.to_pickle('processed_train.pkl')